# 투표 기반 분류기(Voting Classifier)

### 직접 투표(hard voting)
직접투표(hard voting)이란, 다수결 투표처럼, 여러 분류기들이 예측한 결과에서, 가장 많이 선택된 클래스를 예측하는 방법을 의미한다.

<br/>
<img src="./images/hard_voting.png" alt="hard_voting">
<br/>

*그림 출처 : https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting*

### 큰 수의 법칙(law of large numbers)
앞면이 51%, 뒷면이 49%의 확률로 나오는 동전이 있다고 했을 때, 동전을 1000번 던질 경우, 앞면이 다수가 될 확률은 약 75%가 된다.

이는 이항분포의 PMF(확률질량함수)를 구해 CDF(누적분포함수)를 계산해보면 약 0.747의 결과가 나온다는 것을 알 수 있다. 자세한 계산은 다음과 같다.

확률이 p인 이항분포에서, n번의 시도 중 k번 성공할 확률은 다음과 같다.

$$\dfrac{n}{k}p^k(1-p)^{(n-k)}$$

따라서, 성공 확률이 51%인 동전을 1000번 던져 앞면이 한번만 나올 확률은 다음과 같으며,


$$\dfrac{1000}{1}0.51^1(1-0.51)^{(1000-1)} = 1.6 \times 10^{()-307}$$

이를 1000의 과반 직전인 499까지의 확률을 모두 더한 후, 전체 확률 1에서 빼면, 1000번 던져서 앞면이 더 많이 나올 확률이 된다.

`scipy`를 사용하면 이항분포의 CDF를 계산할 수 있다.

In [1]:
from scipy.stats import binom

print(1-binom.cdf(499, 1000, 0.51))

0.7467502275561786


따라서, 51%의 정확도를 가진 1000개의 분류기로 앙상블을 한다면, 75%의 정확도를 얻을 수 있다. (단, 모든 분류기는 완벽히 독립적이고, 오차에 상관관계가 없어야 함)

앙상블 방법은 predictor가 가능한 서로 독립적일 때 가장 큰 성능을 발휘할 수 있다. 각기 다른 알고리즘으로 학습시킨다면, 서로 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 성능을 높일 수 있다.

사이킷런에서는 `VotingClassifier`를 통해 voting classifier를 구현할 수 있다. 다음은 moons dataset을 이용한 예이다.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

각 분류기와 앙상블한 분류기의 테스트셋에서의 정확도는 다음과 같다.

In [4]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.904


### 간접 투표 (soft voting)

간접 투표(soft voting)이란, 여러 분류기들이 예측한 확률을 합산해서 가장 높은 확률의 class를 예측하는 방법을 의미한다. 

<br/>
<img src="./images/soft_voting.png" alt="soft_voting">
<br/>

*그림 출처 : https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting*

soft voting은 확률이 높은 예측에 더 비중을 두기 때문에, hard voting보다 성능이 높다.

사이킷런에서는 `predict_proba()`가 있는 classifier의 경우 사용이 가능하며, `voting="soft"`를 지정해 주면 된다.

다음은 soft voting의 예이다.

In [5]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
# `SVC`에서 predict_proba()를 사용하기 위해 probability=True를 설정해줌
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    # soft voting 설정 
    voting='soft'
)

voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.912
